멀티 DataFrame
```
https://pandas.pydata.org/docs/user_guide/merging.html
```

우리가 친한 emp, dept 테이블을 데려오자

```
pandas.read_sql(
    sql, 
    con, 
    index_col=None, 
    coerce_float=True, 
    params=None, 
    parse_dates=None, 
    columns=None, 
    chunksize=None
)
```

In [138]:
import numpy as np
import pandas as pd
import cx_Oracle
import warnings
warnings.filterwarnings(action='ignore')

# 데이터 준비 from DB

In [160]:
with cx_Oracle.connect('ai', '0000', 'localhost:1521/XE') as conn:
    if bool(conn) : print('연결 성공')
    else: print('연결 실패')
    
    sql = """ select * from dept """
    dept = pd.read_sql(sql, conn)
    sql = """ select * from emp """
    emp = pd.read_sql(sql, conn, parse_dates=['HIREDATE'])

연결 성공


In [161]:
dept.columns = dept.columns.str.lower()
dept

,deptno,dname,loc
0,10,ACCOUNTING,NEW YORK
1,20,RESEARCH,DALLAS
2,30,SALES,CHICAGO
3,40,OPERATIONS,BOSTON


In [162]:
emp.columns = emp.columns.str.lower()
emp

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1987-07-13,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


## 결측처리

In [163]:
emp.isna().sum()[emp.isna().sum()>0].sort_values(ascending=False)

comm    10
mgr      1
dtype: int64

In [166]:
emp[['mgr','comm']] = emp[['mgr','comm']].fillna(0)
emp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   empno     14 non-null     int64         
 1   ename     14 non-null     object        
 2   job       14 non-null     object        
 3   mgr       14 non-null     float64       
 4   hiredate  14 non-null     datetime64[ns]
 5   sal       14 non-null     float64       
 6   comm      14 non-null     float64       
 7   deptno    14 non-null     int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 1.0+ KB


In [151]:
emp.isna().sum()[emp.isna().sum()>0].sort_values(ascending=False)

Series([], dtype: int64)

## 타입변경

In [186]:
emp[['mgr','sal','comm']] = emp[['mgr','sal','comm']].astype('int')
emp

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7369,SMITH,CLERK,7902,1980-12-17,800,0,20
1,7499,ALLEN,SALESMAN,7698,1981-02-20,1600,300,30
2,7521,WARD,SALESMAN,7698,1981-02-22,1250,500,30
3,7566,JONES,MANAGER,7839,1981-04-02,2975,0,20
4,7654,MARTIN,SALESMAN,7698,1981-09-28,1250,1400,30
5,7698,BLAKE,MANAGER,7839,1981-05-01,2850,0,30
6,7782,CLARK,MANAGER,7839,1981-06-09,2450,0,10
7,7788,SCOTT,ANALYST,7566,1987-07-13,3000,0,20
8,7839,KING,PRESIDENT,0,1981-11-17,5000,0,10
9,7844,TURNER,SALESMAN,7698,1981-09-08,1500,0,30


# 합치기 concat
```
pd.concat(데이터프레임리스트, axis=0, join="outer")
```
```
pd.concat(
    objs,
    axis=0,
    join="outer", # 디폴트: 풀 아우터 조인
    ignore_index=False, # 인덱스 무시
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,
)
```
<font color=red>중요중요 빅데이터도 짧은시간에 처리

In [187]:
pd.concat([emp, dept], axis=0)

,empno,ename,job,mgr,hiredate,sal,comm,deptno,dname,loc
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20,NaN,NaN
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30,NaN,NaN
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30,NaN,NaN
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20,NaN,NaN
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30,NaN,NaN
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30,NaN,NaN
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10,NaN,NaN
7,7788.0,SCOTT,ANALYST,7566.0,1987-07-13,3000.0,0.0,20,NaN,NaN
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10,NaN,NaN
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30,NaN,NaN


In [188]:
pd.concat([emp, dept], axis=1)

,empno,ename,job,mgr,hiredate,sal,comm,deptno,deptno,dname,loc
0,7369,SMITH,CLERK,7902,1980-12-17,800,0,20,10.0,ACCOUNTING,NEW YORK
1,7499,ALLEN,SALESMAN,7698,1981-02-20,1600,300,30,20.0,RESEARCH,DALLAS
2,7521,WARD,SALESMAN,7698,1981-02-22,1250,500,30,30.0,SALES,CHICAGO
3,7566,JONES,MANAGER,7839,1981-04-02,2975,0,20,40.0,OPERATIONS,BOSTON
4,7654,MARTIN,SALESMAN,7698,1981-09-28,1250,1400,30,NaN,NaN,NaN
5,7698,BLAKE,MANAGER,7839,1981-05-01,2850,0,30,NaN,NaN,NaN
6,7782,CLARK,MANAGER,7839,1981-06-09,2450,0,10,NaN,NaN,NaN
7,7788,SCOTT,ANALYST,7566,1987-07-13,3000,0,20,NaN,NaN,NaN
8,7839,KING,PRESIDENT,0,1981-11-17,5000,0,10,NaN,NaN,NaN
9,7844,TURNER,SALESMAN,7698,1981-09-08,1500,0,30,NaN,NaN,NaN


# 병합 merge
```
pd.merge(왼쪽데이터프레임,오른쪽데이터프레임, how='outer', on=컬럼명)
```
```
pd.merge(
    left,
    right,
    how="inner", # 디폴트 이너조인
    on=None,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)
```
옵션 how

inner : 겹친키만

left  : 왼쪽키 기준

right : 오른쪽키 기준

outer : 키가 겹치든 안겹치든 싹 다

## 이너조인
```
select * from emp e, dept d where e.deptno=d.deptno;
```
```
select * from emp e inner join dept d on e.deptno=d.deptno;
```

In [189]:
pd.merge(emp, dept, on='deptno', how='inner')

,empno,ename,job,mgr,hiredate,sal,comm,deptno,dname,loc
0,7369,SMITH,CLERK,7902,1980-12-17,800,0,20,RESEARCH,DALLAS
1,7566,JONES,MANAGER,7839,1981-04-02,2975,0,20,RESEARCH,DALLAS
2,7788,SCOTT,ANALYST,7566,1987-07-13,3000,0,20,RESEARCH,DALLAS
3,7876,ADAMS,CLERK,7788,1987-07-13,1100,0,20,RESEARCH,DALLAS
4,7902,FORD,ANALYST,7566,1981-12-03,3000,0,20,RESEARCH,DALLAS
5,7499,ALLEN,SALESMAN,7698,1981-02-20,1600,300,30,SALES,CHICAGO
6,7521,WARD,SALESMAN,7698,1981-02-22,1250,500,30,SALES,CHICAGO
7,7654,MARTIN,SALESMAN,7698,1981-09-28,1250,1400,30,SALES,CHICAGO
8,7698,BLAKE,MANAGER,7839,1981-05-01,2850,0,30,SALES,CHICAGO
9,7844,TURNER,SALESMAN,7698,1981-09-08,1500,0,30,SALES,CHICAGO


## 아우터조인(라이트)
```
select * from emp e, dept d where e.deptno(+)=d.deptno;
```
```
select * from emp e RIGHT OUTER join dept d on e.deptno=d.deptno;
```

In [190]:
pd.merge(emp, dept, on='deptno', how='right')

,empno,ename,job,mgr,hiredate,sal,comm,deptno,dname,loc
0,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10,ACCOUNTING,NEW YORK
1,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10,ACCOUNTING,NEW YORK
2,7934.0,MILLER,CLERK,7782.0,1982-01-23,1300.0,0.0,10,ACCOUNTING,NEW YORK
3,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20,RESEARCH,DALLAS
4,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20,RESEARCH,DALLAS
5,7788.0,SCOTT,ANALYST,7566.0,1987-07-13,3000.0,0.0,20,RESEARCH,DALLAS
6,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20,RESEARCH,DALLAS
7,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,0.0,20,RESEARCH,DALLAS
8,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30,SALES,CHICAGO
9,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30,SALES,CHICAGO


## 풀아우터조인
```
오라클 구현 불가
select * from emp e, dept d where e.deptno(+)=d.deptno(+);
오라클 구현 불가
```
```
select * from emp e full OUTER join dept d on e.deptno=d.deptno;
```

In [218]:
# 연습을 위해 50번 부서를 갖는 사원정보 추가
# emp.loc[14] = [9999, 'MY', 'MASTER', 0, '1980-12-17', 2000, 0, 50]
# emp = emp.drop(14)
# emp

add_emp_list = [[8800, 'TEMPUSER1', 'CLERK', 7902,'1980-12-17', 2000, 100, 50],
           [8801, 'TEMPUSER2', 'CLERK', 7902,'1980-12-17', 2000, 100, 50]]
add_emp_df = pd.DataFrame(add_emp_list, columns=emp.columns)
emp_ = pd.concat([emp, add_emp_df], ignore_index=True)
emp_

,empno,ename,job,mgr,hiredate,sal,comm,deptno
0,7369,SMITH,CLERK,7902,1980-12-17 00:00:00,800,0,20
1,7499,ALLEN,SALESMAN,7698,1981-02-20 00:00:00,1600,300,30
2,7521,WARD,SALESMAN,7698,1981-02-22 00:00:00,1250,500,30
3,7566,JONES,MANAGER,7839,1981-04-02 00:00:00,2975,0,20
4,7654,MARTIN,SALESMAN,7698,1981-09-28 00:00:00,1250,1400,30
5,7698,BLAKE,MANAGER,7839,1981-05-01 00:00:00,2850,0,30
6,7782,CLARK,MANAGER,7839,1981-06-09 00:00:00,2450,0,10
7,7788,SCOTT,ANALYST,7566,1987-07-13 00:00:00,3000,0,20
8,7839,KING,PRESIDENT,0,1981-11-17 00:00:00,5000,0,10
9,7844,TURNER,SALESMAN,7698,1981-09-08 00:00:00,1500,0,30


In [219]:
pd.merge(emp_, dept, on='deptno', how='outer')

,empno,ename,job,mgr,hiredate,sal,comm,deptno,dname,loc
0,7369.0,SMITH,CLERK,7902.0,1980-12-17 00:00:00,800.0,0.0,20,RESEARCH,DALLAS
1,7566.0,JONES,MANAGER,7839.0,1981-04-02 00:00:00,2975.0,0.0,20,RESEARCH,DALLAS
2,7788.0,SCOTT,ANALYST,7566.0,1987-07-13 00:00:00,3000.0,0.0,20,RESEARCH,DALLAS
3,7876.0,ADAMS,CLERK,7788.0,1987-07-13 00:00:00,1100.0,0.0,20,RESEARCH,DALLAS
4,7902.0,FORD,ANALYST,7566.0,1981-12-03 00:00:00,3000.0,0.0,20,RESEARCH,DALLAS
5,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20 00:00:00,1600.0,300.0,30,SALES,CHICAGO
6,7521.0,WARD,SALESMAN,7698.0,1981-02-22 00:00:00,1250.0,500.0,30,SALES,CHICAGO
7,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28 00:00:00,1250.0,1400.0,30,SALES,CHICAGO
8,7698.0,BLAKE,MANAGER,7839.0,1981-05-01 00:00:00,2850.0,0.0,30,SALES,CHICAGO
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08 00:00:00,1500.0,0.0,30,SALES,CHICAGO


# 조인 join
```
쓰지마 머지 ㄱㄱ
```